In [13]:
import pandas as pd
d_estabelecimento = pd.read_csv(r"C:\Users\ABRASEL NACIONAL\Documents\GitHub\ETL_CNPJ\Bases_EMPRESAS\LISTA_EMPRESAS_K03200Y1.csv", sep=';', nrows=1000)
d_estabelecimento.sort_index(ascending=True)

,CNPJ_BASE,RAZAO_SOCIAL
0,0,BANCO DO BRASIL SA
1,1,ASSOCIACAO DE AMIGOS DE BAIRRO DO CONJ PAULISTANO
2,2,WM&R EMPREITEIRA DE CONSTRUCAO CIVIL LIMITADA
3,3,CASA CARIDADE LUZETE ROBERTA DE MORAIS CONJ PA...
4,4,ASSOCIACAO DO PEQUENO ADOLECENTE DO CONJ PAULI...
...,...,...
995,1027,MANOEL MESSIAS DA CONCEICAO SILVA
996,1028,COMERCIAL MARCLAU LTDA
997,1029,ANTONIO SERGIO PINHEIRO CAMPOS
998,1030,JOSE RIBEIRO BIJOUTERIAS


In [ ]:
import pandas as pd
import sqlalchemy
dados = pd.read_parquet('br_base\Restaurantes e similares.gzip')
dados.columns

In [ ]:
import duckdb

# Criando um objeto duckdb
duck = duckdb.connect()

# consulta
query = """
SELECT DATA_SITUACAO_CADASTRAL, 
COUNT(*) as TOTAL
FROM dados 
WHERE CNAE_DESCRICAO = 'RESTAURANTES E SIMILARES'
GROUP BY DATA_SITUACAO_CADASTRAL;
"""

# Exemplo de consulta SQL usando duckdb
resultado = duck.query(query).df()

resultado

In [ ]:
ultima_alteracao = duck.query("""
SELECT DATA_SITUACAO_CADASTRAL 
FROM dados
ORDER BY DATA_SITUACAO_CADASTRAL DESC
LIMIT 1;
""").fetchone()[0]

ultima_alteracao